In [309]:
import sys
sys.path.insert(1, '../src/')
import config
import pandas as pd
import matplotlib.pyplot as plt
import datetime

%matplotlib inline
import plotly.offline as pyoff
import plotly.graph_objs as go
from operator import attrgetter


Let's take a quick look at the data

In [310]:
df = pd.read_csv(config.RETAIL_FILE)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [311]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


Mostly everythig in the raw form is observed and except for the customer ID. We'll look what it looks like and whether they can be inferred?(for example if we see a similar invoice number ).

In [312]:
from collections import Counter
# Most common country is UK by far it even dwarves the second country.
Counter(df['Country'])

Counter({'United Kingdom': 495478,
         'France': 8557,
         'Australia': 1259,
         'Netherlands': 2371,
         'Germany': 9495,
         'Norway': 1086,
         'EIRE': 8196,
         'Switzerland': 2002,
         'Spain': 2533,
         'Poland': 341,
         'Portugal': 1519,
         'Italy': 803,
         'Belgium': 2069,
         'Lithuania': 35,
         'Japan': 358,
         'Iceland': 182,
         'Channel Islands': 758,
         'Denmark': 389,
         'Cyprus': 622,
         'Sweden': 462,
         'Austria': 401,
         'Israel': 297,
         'Finland': 695,
         'Bahrain': 19,
         'Greece': 146,
         'Hong Kong': 288,
         'Singapore': 229,
         'Lebanon': 45,
         'United Arab Emirates': 68,
         'Saudi Arabia': 10,
         'Czech Republic': 30,
         'Canada': 151,
         'Unspecified': 446,
         'Brazil': 32,
         'USA': 291,
         'European Community': 61,
         'Malta': 127,
         'RSA': 58})

Customer id might be a number but the number is just an identifier, so it should not be float let's make it a string(or int)


In [313]:
import numpy as np

In [314]:
# df.loc[:,'CustomerID'] = 
df.loc[:,'customer_id'] = df['CustomerID'].apply(lambda x: str(np.int(x)) if pd.notna(x) else np.nan)

The date column also needs to be formatted as a date.

In [315]:
import datetime
df.loc[:,'date'] = df['InvoiceDate'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y %H:%M'))
#or just
#pd.to_datetime(df['InvoiceDate'])

In [316]:
# take a look at missing customers, and whether we can do any imputations
df.loc[pd.isna(df['customer_id'])]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,customer_id,date
622,536414,22139,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom,NaN,2010-12-01 11:52:00
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom,NaN,2010-12-01 14:32:00
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom,NaN,2010-12-01 14:32:00
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom,NaN,2010-12-01 14:32:00
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom,NaN,2010-12-01 14:32:00
...,...,...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom,NaN,2011-12-09 10:26:00
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom,NaN,2011-12-09 10:26:00
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom,NaN,2011-12-09 10:26:00
541539,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom,NaN,2011-12-09 10:26:00


We see quite some things that we have to decide how we want to process them for example:
- StockCode: DOT
- empty description

Let's see if people with a missing customer id also appear with other customer ids' but same invoice number

In [317]:
unique_invoiced_missing_customers = df.loc[pd.isna(df['customer_id'])]['InvoiceNo'].unique()
df.loc[(df['InvoiceNo'].isin(unique_invoiced_missing_customers)) & (pd.notna(df['customer_id']))]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,customer_id,date


No! nothing we can do with guesing who they are.

(we may want to get rid of them)

But given this data what is it that matters for a company in terms of monitoring their metrics?

A good starting point is with the *monetary revenues* over a period of time.

Take **monthly revenues**:

In [319]:
# first sort df by date
df = df.sort_values(by='date')
df.loc[:,'revenue'] = df['UnitPrice']*df['Quantity']

In [56]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,customer_id,revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,2010-12-01 08:26:00,17850,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01 08:26:00,17850,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,2010-12-01 08:26:00,17850,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01 08:26:00,17850,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01 08:26:00,17850,20.34
...,...,...,...,...,...,...,...,...,...,...,...
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,2011-12-09 12:50:00,12680,16.60
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,2011-12-09 12:50:00,12680,10.20
541900,581587,22730,ALARM CLOCK BAKELIKE IVORY,4,12/9/2011 12:50,3.75,12680.0,France,2011-12-09 12:50:00,12680,15.00
541894,581587,22631,CIRCUS PARADE LUNCH BOX,12,12/9/2011 12:50,1.95,12680.0,France,2011-12-09 12:50:00,12680,23.40


In [320]:
monthly_revenues = df.resample('MS', on='date')['revenue'].agg('sum').reset_index() # MS=Month start
monthly_revenues.rename(columns = {'date':'year_month'}, inplace=True)
monthly_revenues

,year_month,revenue
0,2010-12-01,748957.020
1,2011-01-01,560000.260
2,2011-02-01,498062.650
3,2011-03-01,683267.080
4,2011-04-01,493207.121
5,2011-05-01,723333.510
6,2011-06-01,691123.120
7,2011-07-01,681300.111
8,2011-08-01,682680.510
9,2011-09-01,1019687.622


`Plotly` is very good for *interactive* plots.
- It provides seamless web hosting capabilities for your graphs. 
- if needed you can definitely use Plotly entirely offline, rendering interactive plots directly inside a notebook plus capabilities to export them to (still interactive) HTML files.


In [88]:
# !pip install plotly
# !pip install cufflinks

In [89]:
# !pip install ipywidgets

In [90]:
# !jupyter nbextension enable --py widgetsnbextension

In [323]:


go_plot = [
    go.Scatter(x = monthly_revenues['year_month'], y =monthly_revenues['revenue'])
]

go_layout = go.Layout(
    xaxis = {'type':'category'},
    title = 'Monthly Revenue'
)

fig = go.Figure(data = go_plot, layout = go_layout)

pyoff.iplot(fig)

we see a truncation at the end of our data.

Let's check **monthly growth rate**

In [324]:
monthly_revenues.loc[:,'pct_change'] = monthly_revenues['revenue'].pct_change()

go_plot = [
    go.Scatter(
        x = monthly_revenues.iloc[:-1]['year_month'], 
        y =monthly_revenues.iloc[:-1]['pct_change'])
]

go_layout = go.Layout(
    xaxis = {'type':'category'},
    title = 'Monthly Revenue Growth Rate'
)

fig = go.Figure(data = go_plot, layout = go_layout)

pyoff.iplot(fig)

We see 36.5% growth in December, but -27.8% in April. Why?

Let's check monthly active customers.

Let's focus also on the customers in the UK as they build up the main population.


In [325]:
uk_df = df.query('Country=="United Kingdom"').reset_index(drop=True)


In [330]:
uk_active_monthly_users = uk_df.resample('MS', on='date')[['customer_id']].nunique().reset_index()
uk_active_monthly_users = uk_active_monthly_users.rename(columns = {'date':'year_month', 'customer_id':'nunique_active_users'})
uk_active_monthly_users.loc[:,'pct_change'] = uk_active_monthly_users['nunique_active_users'].pct_change()

In [327]:
go_plot = [
    go.Bar(
        x = uk_active_monthly_users.iloc[:-1]['year_month'], 
        y =uk_active_monthly_users.iloc[:-1]['pct_change'])
]

go_layout = go.Layout(
    xaxis = {'type':'category'},
    title = 'Monthly Acvtive User Growth Rate'
)

fig = go.Figure(data = go_plot, layout = go_layout)

pyoff.iplot(fig)

We see that the active customers dropped in April by 11.5%.

Let's Also check the trend of orders counts.

But since we are repeating this behavior let's make a function.



In [328]:
def df_monthly_col(df, target_col,agg_function, target_name, window = 'MS', date_col = 'date', date_name = 'year_month'):
    df_new = df.resample(window, on=date_col)[[target_col]].agg(agg_function).reset_index()
    df_new = df_new.rename(columns = {date_col:date_name, target_col:target_name})
    df_new.loc[:,'pct_change'] = df_new[target_name].pct_change()
    return df_new

In [331]:
uk_df = df.query('Country=="United Kingdom"').reset_index(drop=True)
uk_active_monthly_users = df_monthly_col(uk_df, target_col='customer_id', agg_function='nunique', target_name = 'nunique_active_users')
uk_active_monthly_users

,year_month,nunique_active_users,pct_change
0,2010-12-01,871,NaN
1,2011-01-01,684,-0.214696
2,2011-02-01,714,0.043860
3,2011-03-01,923,0.292717
4,2011-04-01,817,-0.114843
5,2011-05-01,985,0.205630
6,2011-06-01,943,-0.042640
7,2011-07-01,899,-0.046660
8,2011-08-01,867,-0.035595
9,2011-09-01,1177,0.357555


Let's Also check the trend of orders counts.


In [333]:
uk_order_counts = df_monthly_col(uk_df, target_col='Quantity', agg_function='sum', target_name = 'order_count')
uk_order_counts

,year_month,order_count,pct_change
0,2010-12-01,298101,NaN
1,2011-01-01,237381,-0.203689
2,2011-02-01,225641,-0.049456
3,2011-03-01,279843,0.240213
4,2011-04-01,257666,-0.079248
5,2011-05-01,306452,0.189338
6,2011-06-01,258522,-0.156403
7,2011-07-01,324129,0.253777
8,2011-08-01,319804,-0.013343
9,2011-09-01,458490,0.433659


In [334]:
go_plot1 = [
    go.Bar(
        x = uk_order_counts.iloc[:-1]['year_month'], 
        y =uk_order_counts.iloc[:-1]['order_count'])
]

go_layout1 = go.Layout(
    xaxis = {'type':'category'},
    title = 'Monthly Order Counts'
)

fig1 = go.Figure(data = go_plot1, layout = go_layout1)


go_plot2 = [
    go.Bar(
        x = uk_order_counts.iloc[:-1]['year_month'], 
        y =uk_order_counts.iloc[:-1]['pct_change'])
]

go_layout2 = go.Layout(
    xaxis = {'type':'category'},
    title = 'Monthly Order Counts Growth Rate'
)

fig2 = go.Figure(data = go_plot2, layout = go_layout2)

pyoff.iplot(fig1)
pyoff.iplot(fig2)

We see order count also declined in April.

Let's now check the **average revenue per order**

In [335]:
uk_mean_revenue_df = df_monthly_col(uk_df, target_col='revenue', agg_function='mean', target_name = 'mean_revenue')
uk_mean_revenue_df

,year_month,mean_revenue,pct_change
0,2010-12-01,16.865860,NaN
1,2011-01-01,13.614680,-0.192767
2,2011-02-01,16.093027,0.182035
3,2011-03-01,16.716166,0.038721
4,2011-04-01,15.773380,-0.056400
5,2011-05-01,17.713823,0.123020
6,2011-06-01,16.714748,-0.056401
7,2011-07-01,15.723497,-0.059304
8,2011-08-01,17.315899,0.101275
9,2011-09-01,18.931723,0.093315


In [336]:
def interactive_plot(new_df,target_col, title_1, title_2):
    go_plot1 = [
    go.Bar(
        x = new_df.iloc[:-1]['year_month'], 
        y =new_df.iloc[:-1][target_col])
    ]

    go_layout1 = go.Layout(
        xaxis = {'type':'category'},
        title = title_1
    )

    fig1 = go.Figure(data = go_plot1, layout = go_layout1)


    go_plot2 = [
        go.Bar(
            x = new_df.iloc[:-1]['year_month'], 
            y =new_df.iloc[:-1]['pct_change'])
    ]

    go_layout2 = go.Layout(
        xaxis = {'type':'category'},
        title = title_2
    )

    fig2 = go.Figure(data = go_plot2, layout = go_layout2)

    pyoff.iplot(fig1)
    pyoff.iplot(fig2)

In [337]:
interactive_plot(uk_mean_revenue_df,'mean_revenue', title_1 = 'Monthly Average Revenue Per Order', title_2 = 'Monthly Average Revenue Per Order Growth Rate')


2 more metrics to look at:
- New Customer Ratio: a good indicator of if we are losing our existing customers or unable to attract new ones
- Retention Rate: King of the metrics. Indicates how many customers we retain over specific time window. We will be showing examples for monthly retention rate and cohort based retention rate.

- One way to recognize the progression of a customer purchase is to sort the data by date and give number to them:
- anothe way is to group customers and find the minimum of their date activity.

In [338]:
uk_first_purchase_customers = uk_df.groupby(['customer_id'])['date'].min().reset_index()
uk_first_purchase_customers.columns = ['customer_id','first_purchase_date']
uk_first_purchase_customers

,customer_id,first_purchase_date
0,12346,2011-01-18 10:01:00
1,12747,2010-12-05 15:38:00
2,12748,2010-12-01 12:48:00
3,12749,2011-05-10 15:25:00
4,12820,2011-01-17 12:34:00
...,...,...
3945,18280,2011-03-07 09:52:00
3946,18281,2011-06-12 10:53:00
3947,18282,2011-08-05 13:35:00
3948,18283,2011-01-06 14:14:00


In [341]:
uk_customer_type_df = uk_df.merge(uk_first_purchase_customers,  on='customer_id')   
uk_customer_type_df.loc[:,'customer_type'] = 'Existing'
uk_customer_type_df.loc[uk_customer_type_df['first_purchase_date'] == uk_customer_type_df['date'], 'customer_type'] = 'New'
uk_customer_type_df.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,customer_id,date,revenue,first_purchase_date,customer_type
248981,550890,84836,ZINC METAL HEART DECORATION,4,4/21/2011 12:00,1.25,14711.0,United Kingdom,14711,2011-04-21 12:00:00,5.00,2011-03-22 13:32:00,Existing
111963,550989,22729,ALARM CLOCK BAKELIKE ORANGE,2,4/21/2011 18:22,3.75,13969.0,United Kingdom,13969,2011-04-21 18:22:00,7.50,2010-12-09 19:32:00,Existing
229089,566988,22350,ILLUSTRATED CAT BOWL,18,9/16/2011 10:56,2.55,16709.0,United Kingdom,16709,2011-09-16 10:56:00,45.90,2011-03-03 10:55:00,Existing
222258,558044,23197,VEGETABLE MAGNETIC SHOPPING LIST,12,6/24/2011 13:29,1.45,13870.0,United Kingdom,13870,2011-06-24 13:29:00,17.40,2011-02-25 09:09:00,Existing
4390,573572,23357,HOT WATER BOTTLE SEX BOMB,3,10/31/2011 14:03,4.95,14527.0,United Kingdom,14527,2011-10-31 14:03:00,14.85,2010-12-01 09:41:00,Existing
214771,544278,20727,LUNCH BAG BLACK SKULL.,10,2/17/2011 12:01,1.65,15382.0,United Kingdom,15382,2011-02-17 12:01:00,16.50,2011-02-17 12:01:00,New
318066,573576,20682,RED RETROSPOT CHILDRENS UMBRELLA,1,10/31/2011 14:09,6.63,14096.0,United Kingdom,14096,2011-10-31 14:09:00,6.63,2011-08-30 10:49:00,Existing
354376,576076,22386,JUMBO BAG PINK POLKADOT,1,11/13/2011 16:18,2.08,14382.0,United Kingdom,14382,2011-11-13 16:18:00,2.08,2011-11-13 16:18:00,New
165436,562593,22097,SWALLOW SQUARE TISSUE BOX,3,8/7/2011 15:43,0.39,16549.0,United Kingdom,16549,2011-08-07 15:43:00,1.17,2011-01-09 14:03:00,Existing
56221,552271,22568,FELTCRAFT CUSHION OWL,1,5/8/2011 12:00,3.75,13069.0,United Kingdom,13069,2011-05-08 12:00:00,3.75,2010-12-05 11:02:00,Existing


In [342]:
uk_customer_type_revenues = uk_customer_type_df.groupby([pd.Grouper(key = 'date', freq = 'MS'), 'customer_type'])['revenue'].sum().reset_index()
uk_customer_type_revenues

,date,customer_type,revenue
0,2010-12-01,Existing,193566.620
1,2010-12-01,New,290233.120
2,2011-01-01,Existing,135707.280
3,2011-01-01,New,216274.000
4,2011-02-01,Existing,234696.860
5,2011-02-01,New,114156.770
6,2011-03-01,Existing,306383.090
7,2011-03-01,New,150534.780
8,2011-04-01,Existing,277852.790
9,2011-04-01,New,98891.621


In [343]:
cond_new = uk_customer_type_revenues.loc[uk_customer_type_revenues['customer_type'] == 'New']
cond_ex = uk_customer_type_revenues.loc[uk_customer_type_revenues['customer_type'] == 'Existing']


fig = go.Figure(data=[
    go.Bar(name='New Customers', x=cond_new['date'][:-1], y=cond_new['revenue'].values[:-1]),
    go.Bar(name='Existing Customers', x=cond_ex['date'][:-1], y=cond_ex['revenue'].values[:-1]),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

We see a surge in the revenues from existing customers, but a downward trend in the revenues from the new customers. It seems like the existing customers may have acted loyally, but a burden for new customers to spend in their first month visit.

Let's take a look at new customers ratio:

In [344]:
uk_customer_type_df_counts = uk_customer_type_df.groupby([pd.Grouper(key = 'date', freq = 'MS'), 'customer_type'])['customer_id'].nunique().reset_index()
uk_customer_type_df_counts.columns = ['date', 'customer_type', 'unique_customers']
#
df_new_customers = uk_customer_type_df_counts.loc[uk_customer_type_df_counts['customer_type'] == 'New'].reset_index()
df_old_customers = uk_customer_type_df_counts.loc[uk_customer_type_df_counts['customer_type'] == 'Existing'].reset_index()
df_new_customers.loc[:,'ratio'] = df_new_customers['unique_customers'] / df_old_customers['unique_customers']
df_new_customers.loc[:,'pct_change'] = df_new_customers['ratio'].pct_change()
df_new_customers.reset_index(drop=True, inplace=True)
df_new_customers

,index,date,customer_type,unique_customers,ratio,pct_change
0,1,2010-12-01,New,871,2.655488,NaN
1,3,2011-01-01,New,362,0.885086,-0.666696
2,5,2011-02-01,New,339,0.770455,-0.129514
3,7,2011-03-01,New,408,0.685714,-0.109987
4,9,2011-04-01,New,276,0.458472,-0.331395
5,11,2011-05-01,New,252,0.317380,-0.307743
6,13,2011-06-01,New,207,0.267442,-0.157346
7,15,2011-07-01,New,172,0.227513,-0.149298
8,17,2011-08-01,New,140,0.185430,-0.184968
9,19,2011-09-01,New,275,0.288562,0.556176


In [345]:

go_plot = [
    go.Bar(
        x = df_new_customers['date'][:-1], 
        y=df_new_customers['ratio'].values[:-1]
    )
    ]

go_layout = go.Layout(
    xaxis = {'type':'category'},
    title = 'New Customer Ratio'
)
fig = go.Figure(data=go_plot, layout=go_layout)
# Change the bar mode
pyoff.iplot(fig)

In [346]:
go_plot = [
    go.Bar(
        x = df_new_customers['date'][:-1], 
        y=df_new_customers['pct_change'].values[:-1]
    )
    ]

go_layout = go.Layout(
    xaxis = {'type':'category'},
    title = 'New Customer Ratio Percent Change'
)
fig = go.Figure(data=go_plot, layout=go_layout)
# Change the bar mode
pyoff.iplot(fig)

We can also take a look at **retention rate**, that is retained customers from the previous month

Retention: retained from last month/active customers that month

We can do this three ways:
- pivot tables
- groupby 
- cross tabs

In [404]:
#define active per month

uk_active_customers_revenues = uk_df.groupby([pd.Grouper(key = 'date', freq = 'MS'), 'customer_id'])[['revenue']].sum().reset_index()
uk_active_customers_revenues

,date,customer_id,revenue
0,2010-12-01,12747,706.27
1,2010-12-01,12748,4177.68
2,2010-12-01,12826,155.00
3,2010-12-01,12829,85.75
4,2010-12-01,12838,390.79
...,...,...,...
12325,2011-12-01,18245,894.25
12326,2011-12-01,18272,367.88
12327,2011-12-01,18273,51.00
12328,2011-12-01,18282,77.84


In [406]:
uk_crosstab_date_customer = pd.crosstab(uk_active_customers_revenues['customer_id'], uk_active_customers_revenues['date'])
uk_crosstab_date_customer

date,2010-12-01,2011-01-01,2011-02-01,2011-03-01,2011-04-01,2011-05-01,2011-06-01,2011-07-01,2011-08-01,2011-09-01,2011-10-01,2011-11-01,2011-12-01
customer_id,,,,,,,,,,,,,
12346,0,1,0,0,0,0,0,0,0,0,0,0,0
12747,1,1,0,1,0,1,1,0,1,0,1,1,1
12748,1,1,1,1,1,1,1,1,1,1,1,1,1
12749,0,0,0,0,0,1,0,0,1,0,0,1,1
12820,0,1,0,0,0,0,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,0,0,0,1,0,0,0,0,0,0,0,0,0
18281,0,0,0,0,0,0,1,0,0,0,0,0,0
18282,0,0,0,0,0,0,0,0,1,0,0,0,1


We can sum up the rows for each month, let's also ignore the first month.

In [358]:
date_cols = uk_crosstab_date_customer.columns
retentions = []
for i, month in enumerate(date_cols):
    if i == 0:
        continue;
    retained = {}
    prev_month = date_cols[i-1]
    retained['date'] = month
    retained['total_user'] = uk_crosstab_date_customer[month].sum()
    retained['retained'] =  uk_crosstab_date_customer.loc[(uk_crosstab_date_customer[prev_month] > 0) & (uk_crosstab_date_customer[month] > 0), month].sum()
    retained['retention_rate'] = retained['retained'] / retained['total_user']
    retained['churn_rate'] = 1.0-retained['retention_rate']
    retentions.append(retained)

retention_df = pd.DataFrame(retentions)    
retention_df

,date,total_user,retained,retention_rate,churn_rate
0,2011-01-01,684,322,0.470760,0.529240
1,2011-02-01,714,263,0.368347,0.631653
2,2011-03-01,923,305,0.330444,0.669556
3,2011-04-01,817,310,0.379437,0.620563
4,2011-05-01,985,369,0.374619,0.625381
5,2011-06-01,943,417,0.442206,0.557794
6,2011-07-01,899,379,0.421580,0.578420
7,2011-08-01,867,391,0.450980,0.549020
8,2011-09-01,1177,417,0.354291,0.645709
9,2011-10-01,1285,502,0.390661,0.609339


In [359]:
go_plot = [
    go.Scatter(
        x = retention_df['date'][1:-1], 
        y = retention_df['retention_rate'].values[1:-1]
    )
    ]

go_layout = go.Layout(
    xaxis = {'type':'category'},
    title = 'Monthly customer retention rate'
)
fig = go.Figure(data=go_plot, layout=go_layout)
# Change the bar mode
pyoff.iplot(fig)

We observe a jump from June to August and then drop again.

We can also do the same for a cohort,i.e first purchase month_year of customers

In [422]:
# just looking at the incident of purchases 
uk_unique_df = uk_df.drop_duplicates(['customer_id', 'date', 'InvoiceNo']) 

In [427]:
# a simpler of converting to the month
uk_unique_df['order_month'] = uk_unique_df['date'].dt.to_period('M')
# first purchase month as determinant of the cohort, note the transform in groupby 
uk_unique_df['cohort'] = uk_unique_df.groupby('customer_id')['date'].transform('min').dt.to_period('M') 
# we may want to ignore the last month as it is truncated
uk_unique_df.groupby('cohort')['customer_id'].count()

cohort
2010-12    9235
2011-01    2265
2011-02    1652
2011-03    1721
2011-04     973
2011-05     828
2011-06     649
2011-07     461
2011-08     364
2011-09     643
2011-10     613
2011-11     444
2011-12      37
Freq: M, Name: customer_id, dtype: int64

We groupby the starting cohort and the actual order month. And for each one we compute the number of unique customers within those time periods.

In [428]:

uk_cohort = uk_unique_df.groupby(['cohort', 'order_month'])\
    .agg(n_customers=('customer_id', 'nunique'))\
        .reset_index(drop=False)
uk_cohort

,cohort,order_month,n_customers
0,2010-12,2010-12,871
1,2010-12,2011-01,322
2,2010-12,2011-02,291
3,2010-12,2011-03,329
4,2010-12,2011-04,308
...,...,...,...
86,2011-10,2011-11,86
87,2011-10,2011-12,40
88,2011-11,2011-11,296
89,2011-11,2011-12,41


We also look into the number of periods lapsed between the start and purchase periods of a cohort.

To motivate the use of the atribute way of getting the difference in month

In [449]:
print((uk_cohort['order_month'] - uk_cohort['cohort'])[0])
print()
print(type((uk_cohort['order_month'] - uk_cohort['cohort'])[0]))
print()
help((uk_cohort['order_month'] - uk_cohort['cohort'])[0].n)

<0 * MonthEnds>

<class 'pandas._libs.tslibs.offsets.MonthEnd'>

Help on int object:

class int(object)
 |  int([x]) -> integer
 |  int(x, base=10) -> integer
 |  
 |  Convert a number or string to an integer, or return 0 if no arguments
 |  are given.  If x is a number, return x.__int__().  For floating point
 |  numbers, this truncates towards zero.
 |  
 |  If x is not a number or if base is given, then x must be a string,
 |  bytes, or bytearray instance representing an integer literal in the
 |  given base.  The literal can be preceded by '+' or '-' and be surrounded
 |  by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
 |  Base 0 means to interpret the base from the string as an integer literal.
 |  >>> int('0b100', base=0)
 |  4
 |  
 |  Built-in subclasses:
 |      bool
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __and__(self, value, /)
 |      Return self&val

In [462]:
#attrgetter 
uk_cohort['month_distance'] = (uk_cohort['order_month'] - uk_cohort['cohort']).apply(attrgetter('n'))
uk_cohort

,cohort,order_month,n_customers,period_number,month_distance
0,2010-12,2010-12,871,0,0
1,2010-12,2011-01,322,1,1
2,2010-12,2011-02,291,2,2
3,2010-12,2011-03,329,3,3
4,2010-12,2011-04,308,4,4
...,...,...,...,...,...
86,2011-10,2011-11,86,1,1
87,2011-10,2011-12,40,2,2
88,2011-11,2011-11,296,0,0
89,2011-11,2011-12,41,1,1


Now let's make a pivot table to account for the cohort retention:


In [467]:
cohort_pivot = uk_cohort.pivot_table(index = 'cohort',
                                     columns = 'month_distance',
                                     values = 'n_customers')
cohort_pivot

month_distance,0,1,2,3,4,5,6,7,8,9,10,11,12
cohort,,,,,,,,,,,,,
2010-12,871.0,322.0,291.0,329.0,308.0,345.0,327.0,304.0,306.0,346.0,320.0,429.0,238.0
2011-01,362.0,84.0,101.0,89.0,124.0,106.0,95.0,94.0,114.0,127.0,131.0,54.0,NaN
2011-02,339.0,85.0,65.0,95.0,96.0,86.0,88.0,96.0,94.0,106.0,33.0,NaN,NaN
2011-03,408.0,79.0,107.0,88.0,95.0,70.0,107.0,97.0,119.0,38.0,NaN,NaN,NaN
2011-04,276.0,62.0,61.0,60.0,57.0,64.0,64.0,73.0,23.0,NaN,NaN,NaN,NaN
2011-05,252.0,58.0,43.0,43.0,54.0,60.0,67.0,25.0,NaN,NaN,NaN,NaN,NaN
2011-06,207.0,44.0,34.0,51.0,53.0,67.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN
2011-07,172.0,35.0,33.0,40.0,48.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08,140.0,37.0,32.0,36.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We see that as we move in the matrix to the center the numbers get smaller and smaller due to the churn. But we make this even look better.

0.23268698060941828

In [468]:
cohort_sizes = cohort_pivot.iloc[:,0]
cohort_sizes

cohort
2010-12    871.0
2011-01    362.0
2011-02    339.0
2011-03    408.0
2011-04    276.0
2011-05    252.0
2011-06    207.0
2011-07    172.0
2011-08    140.0
2011-09    275.0
2011-10    318.0
2011-11    296.0
2011-12     34.0
Freq: M, Name: 0, dtype: float64

In [471]:
retentions = cohort_pivot.divide(cohort_sizes, axis = 0)
retentions

month_distance,0,1,2,3,4,5,6,7,8,9,10,11,12
cohort,,,,,,,,,,,,,
2010-12,1.0,0.369690,0.334099,0.377727,0.353617,0.396096,0.375431,0.349024,0.351320,0.397245,0.367394,0.492537,0.273249
2011-01,1.0,0.232044,0.279006,0.245856,0.342541,0.292818,0.262431,0.259669,0.314917,0.350829,0.361878,0.149171,NaN
2011-02,1.0,0.250737,0.191740,0.280236,0.283186,0.253687,0.259587,0.283186,0.277286,0.312684,0.097345,NaN,NaN
2011-03,1.0,0.193627,0.262255,0.215686,0.232843,0.171569,0.262255,0.237745,0.291667,0.093137,NaN,NaN,NaN
2011-04,1.0,0.224638,0.221014,0.217391,0.206522,0.231884,0.231884,0.264493,0.083333,NaN,NaN,NaN,NaN
2011-05,1.0,0.230159,0.170635,0.170635,0.214286,0.238095,0.265873,0.099206,NaN,NaN,NaN,NaN,NaN
2011-06,1.0,0.212560,0.164251,0.246377,0.256039,0.323671,0.096618,NaN,NaN,NaN,NaN,NaN,NaN
2011-07,1.0,0.203488,0.191860,0.232558,0.279070,0.110465,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08,1.0,0.264286,0.228571,0.257143,0.135714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The way we read this:
For example for the first cohorot we see that almost a year (11 month distance) the retention of that cohort is about 50% which is very impressive.

In the second cohort we see a huge drop, yet the maximum retained customers for this cohort is around the same time that first cohort had its highest retention, so possibly there is some incentives around that time of the year(for example discounts), as is a pattern we see among all cohorts, and a huge drop after that incentive.
